In [3]:
import numpy as np
import torch
from torch import nn
import nltk
nltk.download('stopwords')
import pandas as pd
from nltk.corpus import stopwords
import torch.optim as optim
from tqdm import tqdm
import os

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
contractions = { 
"n't": "not",
"'ve": "have",
"'d": "would",
"'ll": "will",
"'s": "is",
"'m": "am",
"ma'am": "madam",
"'re": "they are"
}

In [0]:
def clean_text(text, remove_stopwords = False):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

In [0]:
embeddings_index = {}
with open('/content/drive/My Drive/numberbatch-en.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

In [0]:
training_data=[]
with open('/content/drive/My Drive/train.txt','r') as f:
  line=f.readline()
  while(line):
    line=f.readline()
    if(line.strip()==''):
       training_data.append({'sent':'','label':''})
    else: 
      line=line.split()  
      training_data[-1]['sent']+=' '+str(line[0])
      training_data[-1]['label']+=' '+str(line[-1])

validation_data=[]
with open('/content/drive/My Drive/valid.txt','r') as f:
  line=f.readline()
  while(line):
    line=f.readline()
    if(line.strip()==''):
       validation_data.append({'sent':'','label':''})
    else: 
      line=line.split()  
      validation_data[-1]['sent']+=' '+str(line[0])
      validation_data[-1]['label']+=' '+str(line[-1])   

In [0]:
df=pd.DataFrame(training_data)
df_valid=pd.DataFrame(validation_data)

In [0]:
df['label'].replace('',np.nan,inplace=True)
df_valid['label'].replace('',np.nan,inplace=True)

In [11]:
print(df.isna().sum())
print(df_valid.isna().sum())

sent     0
label    2
dtype: int64
sent     0
label    2
dtype: int64


In [0]:
df.dropna(inplace=True)

In [0]:
def count_words(count_dict, text):
    '''Count the number of occurrences of each word in a set of text'''
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1

In [0]:
word_counts = {}

count_words(word_counts, df['sent'].values)

In [15]:
missing_words = 0
threshold = 20

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round(missing_words/len(word_counts),4)*100
            
print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))

Number of words missing from CN: 434
Percent of words that are missing from vocabulary: 1.8399999999999999%


In [16]:
vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

# Special tokens that will be added to our vocab
codes = ["<UNK>","</s>","<s>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_ratio))    

Total number of unique words: 23624
Number of words we will use: 9473
Percent of words we will use: 40.1%


In [0]:
def convert_to_ints(sentence):
    sentence_int = []
    # print(sentence)
    # sentence_int.append(vocab_to_int['<s>'])
    for word in sentence.split():
        if word in vocab_to_int:
            sentence_int.append(vocab_to_int[word])
        else:
            sentence_int.append(vocab_to_int["<UNK>"])
        # sentence_int.append(vocab_to_int["</s>"])        
    return sentence_int

In [0]:
df['encoding']=df['sent'].apply(lambda x:convert_to_ints(x))
df_valid['encoding']=df_valid['sent'].apply(lambda x:convert_to_ints(x))

In [0]:
unique_labels=np.unique(''.join(list(df['label'])).split())
tag_to_index={}
index_to_tag={}
for i,lbl in enumerate(list(unique_labels)):
  tag_to_index[lbl]=i
  index_to_tag[i]=lbl

In [0]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))    

In [0]:
class BiLSTM_CRF(nn.Module):
    def __init__(self,vocab_size,hidden_size,embedding_size,tag_to_index):
        super(BiLSTM_CRF, self).__init__()
        self.target_size=len(tag_to_index)
        self.embed = nn.Embedding(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size // 2,num_layers=2, bidirectional=True)
        self.hidden2tag = nn.Linear(hidden_size, self.target_size)
        self.transitions = nn.Parameter(torch.randn(self.target_size, self.target_size))
        self.start_transition=nn.Parameter(torch.randn(1, self.target_size))
        self.end_transition=nn.Parameter(torch.randn(1, self.target_size))
        # self.hidden=self.init_hidden(hidden_size)
    def _get_lstm_features(self,sentence):
        embed=self.embed(sentence)
        output,hidden=self.lstm(embed.view(len(sentence),1,50))
        output=self.hidden2tag(output)
        return output
    def _forward(self,features):   
        temp= self.start_transition.data+features[0,0,:]
        labels=[]
        lbl=torch.argmax(temp,1)
        # score=torch.max(veterbi_var)
        labels.append(lbl)
        # labels.append(argmax(veterbi_var))
        # score=torch.max(veterbi_var)        
        for indx in range(1,features.shape[0]):
            veterbi_var=[]
            for tag in range(features.shape[2]):                             
                veterbi_var.append(log_sum_exp(temp+self.transitions[:,tag].view(1,9)+features[indx,0,tag].expand(1,9)).view(1,1))
            # print(veterbi_var)                   
            temp=torch.cat(veterbi_var,dim=1)
            # print(temp.shape)
            lbl=torch.argmax(temp,1)
                # score=torch.max(veterbi_var)
            labels.append(lbl)
            # score=torch.max(veterbi_var)   
        temp+=self.end_transition.view(1,-1)
        # print(torch.min(temp))
        score=log_sum_exp(temp)
        return score,labels
    def _score_sentence(self,feats,tags):
        score=feats[0,0,tags[0]]+self.start_transition[0,tags[0]]
        for indx in range(1,feats.shape[0]):
            # print(feats.shape,tags[indx],indx,tags.shape)
            score+=feats[indx,0,tags[indx]]+self.transitions[tags[indx-1],tags[indx]]
        score+=self.end_transition[0,tags[-1]] 
        return score
    def _neg_log_likelihood(self,sentence,targets):
        feats = self._get_lstm_features(sentence)
        forward_score,_=self._forward(feats)
        gold_score = self._score_sentence(feats, targets) 
        # print('forward score {}'.format(forward_score),'gold score {}'.format(gold_score)) 
        loss=forward_score - gold_score     
        return loss
    def _viterbi(self,features):
        # print(self.start_transition.data,features)
        temp= self.start_transition.data.view(1,-1)+features[0,0,:].view(1,-1)
        labels=[]
        # print(temp)
        temp,lbl=torch.max(temp,1)
        # print(temp,lbl)
        labels.append(lbl)     
        for indx in range(1,features.shape[0]):
            veterbi_var=[]
            for tag in range(features.shape[2]):                            
                veterbi_var.append(temp.view(1,1)+self.transitions[lbl[-1],tag].view(1)+features[indx,0,tag].view(1))
            # print(veterbi_var)                   
            temp=torch.cat(veterbi_var,dim=1)
            # lbl=torch.argmax(temp,1)
            temp,lbl=torch.max(temp,1)
            labels.append(lbl)  
        temp=temp.view(1,1)+self.end_transition[0,lbl].view(1,1)
        return temp,labels  
    def _predict(self,sentence):
        feats = self._get_lstm_features(sentence)
        score,labels=self._viterbi(feats)
        return labels

In [26]:
import pickle
from torch.nn.utils import clip_grad_norm_
vocab_size=len(vocab_to_int)
hidden_size=4
embedding_size=50
df_valid.dropna(inplace=True)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model=BiLSTM_CRF(len(vocab_to_int),hidden_size,embedding_size,tag_to_index).to(device)
# if(os.path.exists('/content/drive/My Drive/model.pt')):
#   model.load_state_dict(torch.load('/content/drive/My Drive/model.pt'))
optimizer = optim.SGD(model.parameters(), lr=1e-1, weight_decay=1e-4)
losses=[]
# if(os.path.exists('/content/drive/My Drive/losses.pkl')):
#   with open('/content/drive/My Drive/losses.pkl','rb') as f:
#     losses=pickle.load(f)
for epoch in tqdm(range(500)): 
    i=0
    for index,row in df.iterrows():
        model.zero_grad()
        sent=torch.tensor(row['encoding'],dtype=torch.long).to(device)
        targets = torch.tensor([tag_to_index[t] for t in row['label'].split()], dtype=torch.int32).to(device)
        loss = model._neg_log_likelihood(sent, targets)
        loss.backward()
        # print(loss.data.item())
        clip_grad_norm_(model.parameters(), 1)
        optimizer.step() 
        i+=1
        if(i==10):
          break
    print(loss.data.item())  
        # losses.append(loss)
    # val_loss=0    
    # for index,row in df_valid.iterrows():
    #     model.zero_grad()
    #     sent=torch.tensor(row['encoding'],dtype=torch.long).to(device)
    #     targets = torch.tensor([tag_to_index[t] for t in row['label'].split()], dtype=torch.int32).to(device)
    #     val_loss+= model._neg_log_likelihood(sent, targets).item()    
    # val_loss=val_loss/len(df_valid)
    # print('training loss:{}'.format(loss),'validation loss:{}'.format(val_loss))
    # if(len(losses)>0 and val_loss<min(losses)):
    #   torch.save(model.state_dict(), '/content/drive/My Drive/model.pt')            
    # losses.append(val_loss)


  0%|          | 1/500 [00:01<09:52,  1.19s/it]

71.82562255859375



  0%|          | 2/500 [00:02<09:52,  1.19s/it]

33.57405090332031



  1%|          | 3/500 [00:03<09:52,  1.19s/it]

...


 51%|█████     | 256/500 [05:05<04:48,  1.18s/it]

0.3121337890625



 51%|█████▏    | 257/500 [05:07<04:46,  1.18s/it]

0.25390625


KeyboardInterrupt: ignored

In [0]:
# for index,row in df_valid.iterrows():
#   try:
#     df_valid.iloc[index]['label'].split()
#   except:
#     print(index,df_valid.iloc[index]['label'])  

# print(3465,df_valid.iloc[index])      

3465 sent           
label       NaN
encoding     []
Name: 3466, dtype: object


# Testing

In [0]:
def process_input(file):
    valid_data=[]
    with open(file,'r') as f:
      line=f.readline()
      while(line):
        line=f.readline()
        if(line.strip()==''):
          valid_data.append({'sent':'','label':''})
        else: 
          line=line.split()  
          valid_data[-1]['sent']+=' '+str(line[0])
          valid_data[-1]['label']+=' '+str(line[-1])
    df=pd.DataFrame(valid_data)
    df['label'].replace('',np.nan,inplace=True)
    df.dropna(inplace=True)
    df['encoding']=df['sent'].apply(lambda x:convert_to_ints(x))
    target=[]
    return df    

In [27]:
vocab_size=len(vocab_to_int)
hidden_size=4
embedding_size=50
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model=BiLSTM_CRF(len(vocab_to_int),hidden_size,embedding_size,tag_to_index)
# model.load_state_dict(torch.load('/content/drive/My Drive/model.pt'))
# model.to(device)
valid_df=process_input('/content/drive/My Drive/train.txt')
i=0
for index,row in valid_df.iterrows():
    # print(row['sent'])
    prediction=model._predict(torch.tensor(row['encoding']).to(device))
    print(row['sent'])
    print(row['label'])
    # print(row['encoding'])
    print([index_to_tag[int(t)] for t in prediction])
    # print(row['label'])
    i+=1
    if(i%10==0):
      break

 EU rejects German call to boycott British lamb .
 B-ORG O B-MISC O O O B-MISC O O
['O', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']
 Peter Blackburn
 B-PER I-PER
['O', 'B-PER']
 BRUSSELS 1996-08-22
 B-LOC O
['O', 'O']
 The European Commission said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep .
 O B-ORG I-ORG O O O O O O B-MISC O O O O O B-MISC O O O O O O O O O O O O O O
['O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
 Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer .
 B-LOC O O O O B-ORG I-ORG O O O B-PER I-PER O O O O O O O O O O O B-LOC O O O O O O O
['O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 

In [28]:
index_to_tag

{0: 'B-LOC',
 1: 'B-MISC',
 2: 'B-ORG',
 3: 'B-PER',
 4: 'I-LOC',
 5: 'I-MISC',
 6: 'I-ORG',
 7: 'I-PER',
 8: 'O'}